# 实现工作总结的原型

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 3.47 s, sys: 417 ms, total: 3.89 s
Wall time: 3.54 s


In [2]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.11 ms, sys: 110 µs, total: 1.22 ms
Wall time: 1.01 ms


## 加载数据

In [3]:
%%time

# 模拟一个当天的全体人员工作日志
items=[
"开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引",
"开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整",
"产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询",
"产品部林晓琪，江南皮革厂客服助手项目，和客户开会，沟通需求",
"运维部李泽言，江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题",
"销售部韩雪薇，天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案",
"法务部张峻宁，完成审核了与深圳做大事公司的商务合同",
"开发部郑千，simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误",
]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


In [4]:
from llama_index.core.schema import TextNode

nodes=[ TextNode(text=item, id_=f"node_{index}") for index, item in enumerate(items)]
nodes

[TextNode(id_='node_0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='node_1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='node_2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='no

## 建立摘要索引和向量索引

In [5]:
%%time


from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

CPU times: user 35.2 ms, sys: 1.26 ms, total: 36.4 ms
Wall time: 1.76 s


## 定义查询引擎和元数据

In [6]:
%%time


list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

CPU times: user 33.4 ms, sys: 0 ns, total: 33.4 ms
Wall time: 32.9 ms


In [7]:
%%time

from llama_index.core.tools import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "对工作总结方面的问题很有用。"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于工作相关的具体问题。"
    ),
)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 15.7 µs


### 单选

In [22]:
%%time

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

CPU times: user 161 µs, sys: 18 µs, total: 179 µs
Wall time: 182 µs


In [23]:
response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
)

In [24]:
print(str(response))

产品部，林晓琪：
1. 提交了多数据源融合检索查询的新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部，韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误问题。

开发部，陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了聊天界面markdown格式显示的bug修复工作。

运维部，李泽言：
1. 解决了江南皮革厂客服平台负载过高导致的服务卡顿问题；
2. 进行了系统扩容操作。

销售部，韩雪薇：
1. 准备了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部，张峻宁：
1. 审核并完成了与深圳做大事公司的商务合同。


In [11]:
response = query_engine.query(
    "总结开发部完成的任务。"
    "要求：人员按照如下次序，韩宇轩、陈逸飞、郑千"
    "格式要求："
    "开发部工作总结："
    "- 张三： 1. 解决了xxx；2. 完成了xxx"
    "- 李四： 1. 修正了xxx"
    "..."
)
print(str(response))

开发部工作总结：
- 陈逸飞：1. 修正bug：中文断句不完整；
- 郑千：1. 修正bug：聊天界面markdown格式显示错误。


In [12]:
%%time

response = query_engine.query("按照项目/产品/业务，列出开发部工作完成的情况，列表说明：谁干了什么")
print(str(response))

按照项目（simple-rag产品）和任务分类，以下是开发部的工作情况：

1. **陈逸飞**完成了对中文断句不完整问题的修正。
2. **郑千**解决了聊天界面markdown格式显示错误的问题。
CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 2.08 s


In [13]:
%%time

response = query_engine.query("列出开发部所有人员姓名")
print(str(response))

开发部有两位人员，他们的姓名分别是陈逸飞和韩宇轩。
CPU times: user 17.8 ms, sys: 0 ns, total: 17.8 ms
Wall time: 1.31 s


In [14]:
%%time

response = query_engine.query("张峻宁干了啥")
print(str(response))

张峻宁完成了对与深圳做大事公司的商务合同的审核工作。
CPU times: user 15.8 ms, sys: 2.52 ms, total: 18.3 ms
Wall time: 1.29 s


In [15]:
%%time

response = query_engine.query("simple-rag 产品都完成了哪些事情，包括开发部和产品部。列表说明。")
print(str(response))

简单RAG产品中，开发部韩宇轩实现了新特性：针对文档的摘要索引；开发部陈逸飞修正了bug：中文断句不完整。
CPU times: user 17.5 ms, sys: 501 µs, total: 18 ms
Wall time: 1.85 s


### 多选

In [16]:
%%time

from llama_index.core.selectors import LLMMultiSelector

query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

CPU times: user 195 µs, sys: 0 ns, total: 195 µs
Wall time: 198 µs


In [17]:
%%time

response = query_engine.query("按照项目/产品/业务，列出工作完成的情况，列表说明")
print(str(response))

销售部韩雪薇正在准备标书商务部分，并专注于公司资质方面的文案撰写。这表明她已经完成了与天津化工集团后勤服务平台相关的准备工作。

产品部林晓琪则在进行simple-rag产品的特新需求提交，具体是关于多数据源的融合检索查询功能的需求。这意味着她的工作也已进入完成阶段，正在为产品特性添加新的功能需求。
CPU times: user 13.1 ms, sys: 3.77 ms, total: 16.8 ms
Wall time: 2.67 s


In [18]:
%%time

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
)
print(str(response))

产品部，林晓琪：
1. 提交了多数据源融合检索查询的新特性需求。
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部，韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性。

开发部，陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug。
2. 解决了聊天界面markdown格式显示错误的bug。

运维部，李泽言：
1. 对江南皮革厂客服平台进行了系统扩容，解决了负载过高导致的服务卡顿问题。

销售部，韩雪薇：
1. 准备了天津化工集团后勤服务平台的标书商务部分。
2. 完成了公司资质方面的文案工作。

法务部，张峻宁：
1. 审核并完成了与深圳做大事公司的商务合同。
CPU times: user 13.7 ms, sys: 191 µs, total: 13.9 ms
Wall time: 4.55 s


In [19]:
%%time

response = query_engine.query("法务部完成了啥")
print(str(response))

法务部完成了与深圳做大事公司的商务合同的审核工作。
CPU times: user 30.8 ms, sys: 299 µs, total: 31.1 ms
Wall time: 3.11 s
